# Supervised ANNs for Mutual Information Investigation
## Thomas Possidente

### Imports and Initializations

In [1]:
# ANN Building Imports
import keras
from keras import backend as K
from keras import optimizers, losses
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import tensorflow as tf
from keras.callbacks import LambdaCallback


# Standard Imports
import pandas as pd
import numpy as np

# Value Inits - Specify as needed
num_inputs_per_batch = int(1000)  # number of dummy images in set
size = int(16)          # Dimension of each dummy image should be size*size
RF_size = int(4)        # Dimensions of the RF to be analyzed should be RF_size*RF_size

# Value Inits - Leave these alone
num_of_RFs = int((size*size) / (RF_size*RF_size))
inputs_by_RF = np.empty([(num_inputs_per_batch * size), size])


C:\Users\Tom\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Reading in Data

In [2]:
inputs = pd.read_csv('test.csv')
inputs = inputs.drop('X1', axis = 0) # Taking out col names
inputs = inputs.apply(pd.to_numeric)  # converting to floats

inputs = inputs.values # convert to np ndarray
inputs = inputs.reshape(5000,16,16) # reshape to desired dims (5000 examples, 16*16)
flattened_inputs = inputs.reshape(5000, 16*16)

### Creating Labels

In [3]:
labels = np.zeros(np.shape(flattened_inputs)[0])
storage = np.zeros(np.shape(flattened_inputs))
count = 0

for i in range(np.shape(flattened_inputs)[0]):
    if(~((flattened_inputs[i] == storage).all(1).any())):
        labels[i] = count
        storage[i] = flattened_inputs[i]
        count += 1
    elif((flattened_inputs[i] == storage).all(1).any()):
        print(1)
        label_index = np.where((flattened_inputs[i] == storage).all(1))
        correct_label = labels[label_index]
        storage[i] = flattened_inputs[i]
        labels[i] = correct_label
        
labels = labels.astype(int)
    

In [4]:
labels

array([   0,    1,    2, ..., 4997, 4998, 4999])

### Adding Noise to Inputs

### Building Network

In [10]:
model = Sequential()
model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = (RF_size, RF_size), strides = RF_size, activation = 'relu'))
model.add(Flatten())
model.add(Dense(256, activation='softmax'))

In [15]:
model.compile(optimizer = optimizers.sgd(lr = 0.01), loss = 'categorical_crossentropy')

In [ ]:
model.fit(x = inputs, y = , batch_size = , epochs = )